# Summation and Catastrophic Cancellation

## Objective

Study how rounding errors accumulate in summation and demonstrate catastrophic cancellation:
- Naive summation error growth
- Kahan (compensated) summation
- Pairwise summation
- Catastrophic cancellation examples

## Error Growth in Naive Summation

For naive left-to-right summation of $n$ numbers, the rounding error grows as:

$$|\text{error}| \leq nu|\sum x_i|$$

where $u$ is the unit roundoff. This is a **linear growth** in $n$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')

from utils.floating_point_tools import naive_sum, kahan_sum, pairwise_sum
from utils.error_metrics import relative_error

np.random.seed(42)
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## Experiment 1: Error Growth vs Problem Size

In [ ]:
# Test summation error growth
sizes = np.logspace(2, 7, 20).astype(int)
naive_errors = []
kahan_errors = []
pairwise_errors = []
numpy_errors = []

for n in sizes:
    # Generate random numbers
    arr = np.random.randn(n)
    
    # Compute "exact" sum using higher precision
    exact = np.sum(arr.astype(np.float128))
    
    # Compute sums with different methods
    s_naive = naive_sum(arr)
    s_kahan = kahan_sum(arr)
    s_pairwise = pairwise_sum(arr)
    s_numpy = np.sum(arr)
    
    # Record relative errors
    naive_errors.append(abs(s_naive - exact) / abs(exact))
    kahan_errors.append(abs(s_kahan - exact) / abs(exact))
    pairwise_errors.append(abs(s_pairwise - exact) / abs(exact))
    numpy_errors.append(abs(s_numpy - exact) / abs(exact))

# Plot results
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.loglog(sizes, naive_errors, 'o-', label='Naive', linewidth=2)
plt.loglog(sizes, kahan_errors, 's-', label='Kahan', linewidth=2)
plt.loglog(sizes, pairwise_errors, '^-', label='Pairwise', linewidth=2)
plt.loglog(sizes, numpy_errors, 'd-', label='NumPy', linewidth=2)

# Theoretical bounds
eps = np.finfo(np.float64).eps
plt.loglog(sizes, sizes * eps, 'k--', alpha=0.5, label=r'$n \varepsilon$')
plt.loglog(sizes, np.log2(sizes) * eps, 'k:', alpha=0.5, label=r'$\log_2(n) \varepsilon$')

plt.xlabel('Number of summands (n)')
plt.ylabel('Relative error')
plt.title('Summation Error Growth')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Improvement factor
improvement_kahan = np.array(naive_errors) / np.array(kahan_errors)
improvement_pairwise = np.array(naive_errors) / np.array(pairwise_errors)

plt.semilogx(sizes, improvement_kahan, 's-', label='Kahan vs Naive', linewidth=2)
plt.semilogx(sizes, improvement_pairwise, '^-', label='Pairwise vs Naive', linewidth=2)
plt.xlabel('Number of summands (n)')
plt.ylabel('Error reduction factor')
plt.title('Improvement Over Naive Summation')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../plots/02_summation_error_growth.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"For n = {sizes[-1]:,}:")
print(f"Naive error:     {naive_errors[-1]:.6e}")
print(f"Kahan error:     {kahan_errors[-1]:.6e}")
print(f"Pairwise error:  {pairwise_errors[-1]:.6e}")
print(f"Improvement:     {improvement_kahan[-1]:.1f}x (Kahan)")

## Kahan Summation Algorithm

Kahan's compensated summation maintains a running compensation for lost low-order bits:

```
sum = 0
c = 0  # compensation
for x in array:
    y = x - c
    t = sum + y
    c = (t - sum) - y
    sum = t
```

**Error bound**: $O(\varepsilon) + O(n\varepsilon^2)$ instead of $O(n\varepsilon)$

## Experiment 2: Catastrophic Cancellation

Subtracting nearly equal numbers loses significant digits.

In [ ]:
from utils.floating_point_tools import catastrophic_cancellation_example

# Example: (1 - cos(x)) / x^2 for small x
x_values = np.logspace(-10, -1, 100)

unstable_result = catastrophic_cancellation_example(x_values, method='unstable')
stable_result = catastrophic_cancellation_example(x_values, method='stable')

# Exact value (limit as x -> 0 is 1/2)
exact = 0.5 * np.ones_like(x_values)

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.semilogx(x_values, unstable_result, 'o-', label='Unstable: (1 - cos(x))/x²', markersize=4)
plt.semilogx(x_values, stable_result, 's-', label='Stable: reformulated', markersize=4)
plt.axhline(0.5, color='k', linestyle='--', label='Exact (0.5)')
plt.xlabel('x')
plt.ylabel('Function value')
plt.title('Catastrophic Cancellation Example')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
unstable_error = np.abs(unstable_result - exact) / exact
stable_error = np.abs(stable_result - exact) / exact

plt.loglog(x_values, unstable_error, 'o-', label='Unstable', markersize=4)
plt.loglog(x_values, stable_error, 's-', label='Stable', markersize=4)
plt.xlabel('x')
plt.ylabel('Relative error')
plt.title('Error Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../plots/02_catastrophic_cancellation.png', dpi=150, bbox_inches='tight')
plt.show()

## Experiment 3: Quadratic Formula Stability

In [ ]:
from utils.floating_point_tools import quadratic_formula

# Solve x^2 - 2bx + 1 = 0 for large b
# Exact roots: b ± sqrt(b^2 - 1) ≈ b ± b for large b
b_values = np.logspace(1, 8, 50)

unstable_errors = []
stable_errors = []

for b in b_values:
    a, c = 1.0, 1.0
    
    # Exact roots (using high precision)
    b_hp = np.float128(b)
    disc = b_hp**2 - 4
    r1_exact = (b_hp + np.sqrt(disc)) / 2
    r2_exact = (b_hp - np.sqrt(disc)) / 2
    
    # Unstable formula
    r1_unstable, r2_unstable = quadratic_formula(a, -2*b, c, method='unstable')
    
    # Stable formula
    r1_stable, r2_stable = quadratic_formula(a, -2*b, c, method='stable')
    
    # Compute errors for the smaller root (more sensitive)
    unstable_err = abs(r2_unstable - r2_exact) / abs(r2_exact)
    stable_err = abs(r2_stable - r2_exact) / abs(r2_exact)
    
    unstable_errors.append(unstable_err)
    stable_errors.append(stable_err)

plt.figure(figsize=(10, 6))
plt.loglog(b_values, unstable_errors, 'o-', label='Unstable formula', linewidth=2)
plt.loglog(b_values, stable_errors, 's-', label='Stable formula', linewidth=2)
plt.axhline(np.finfo(np.float64).eps, color='k', linestyle='--', 
            label=r'Machine epsilon', alpha=0.5)
plt.xlabel('Parameter b')
plt.ylabel('Relative error (smaller root)')
plt.title('Quadratic Formula Stability')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('../plots/02_quadratic_formula.png', dpi=150, bbox_inches='tight')
plt.show()

## Experiment 4: log1p and expm1

In [ ]:
from utils.floating_point_tools import log1p_example, expm1_example

x_small = np.logspace(-16, -1, 100)

# log(1 + x)
log1p_unstable = log1p_example(x_small, method='unstable')
log1p_stable = log1p_example(x_small, method='stable')
log1p_exact = np.log1p(x_small.astype(np.float128)).astype(np.float64)

# exp(x) - 1
expm1_unstable = expm1_example(x_small, method='unstable')
expm1_stable = expm1_example(x_small, method='stable')
expm1_exact = np.expm1(x_small.astype(np.float128)).astype(np.float64)

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
log1p_unstable_err = np.abs(log1p_unstable - log1p_exact) / np.abs(log1p_exact)
log1p_stable_err = np.abs(log1p_stable - log1p_exact) / np.abs(log1p_exact)
plt.loglog(x_small, log1p_unstable_err, 'o-', label='log(1+x) unstable', markersize=4)
plt.loglog(x_small, log1p_stable_err, 's-', label='log1p stable', markersize=4)
plt.xlabel('x')
plt.ylabel('Relative error')
plt.title('log(1 + x) for small x')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
expm1_unstable_err = np.abs(expm1_unstable - expm1_exact) / np.abs(expm1_exact)
expm1_stable_err = np.abs(expm1_stable - expm1_exact) / np.abs(expm1_exact)
plt.loglog(x_small, expm1_unstable_err, 'o-', label='exp(x)-1 unstable', markersize=4)
plt.loglog(x_small, expm1_stable_err, 's-', label='expm1 stable', markersize=4)
plt.xlabel('x')
plt.ylabel('Relative error')
plt.title('exp(x) - 1 for small x')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../plots/02_log1p_expm1.png', dpi=150, bbox_inches='tight')
plt.show()

## Key Takeaways

1. **Naive summation**: Error grows linearly with $n$, reaching $O(n\varepsilon)$

2. **Kahan summation**: Reduces error to $O(\varepsilon)$, dramatic improvement for large $n$

3. **Pairwise summation**: Error grows as $O(\log n \cdot \varepsilon)$, good balance

4. **Catastrophic cancellation**: Subtracting nearly equal numbers loses precision

5. **Algorithmic reformulation**: Often the best solution (e.g., log1p, expm1)

6. **Practical impact**: For $n = 10^7$, Kahan can improve accuracy by $10^7$ times!